In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.nn.utils import clip_grad_norm_

torch.manual_seed(1)

In [2]:
import json
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer

[nltk_data] Downloading package punkt to C:\Users\Mihaela
[nltk_data]     Stoycheva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# tokenizer = TweetTokenizer(preserve_case=False)
tok = MWETokenizer([('<', 'unk', '>')], separator = '')

train_data = []
vocabulary = []

with open("data/ptb.train.txt") as f:
    total_sentences = 0
    for i, line in enumerate(f):
        line = line.strip()
        s = word_tokenize(line)
        s = tok.tokenize(s)
        s = ['<sos>'] + s + ['<eos>']
        total_sentences += 1
        train_data.append(s)
        vocabulary += s

vocabulary = sorted(list(set(vocabulary)))
word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

print("total sentences:", total_sentences)

total sentences: 42068


In [4]:
print(vocabulary[0:100])

['#', '$', '&', "'", "'80s", "'d", "'ll", "'m", "'re", "'s", "'ve", '.', '10-year', '100-share', '12-month', '12-year', '13-week', '13th', '14-year-old', '190-point', '190.58-point', '1920s', '1930s', '1950s', '1960s', '1970s', '1980s', '1990s', '19th', '1\\/2-year', '2-for-1', '20-year', '20th', '24-hour', '26-week', '30-day', '30-share', '30-year', '300-a-share', '300-day', '40-year-old', '45-year-old', '500-stock', '52-week', '<eos>', '<sos>', '<unk>', 'N', '\\*', '\\*\\*', 'a', 'a.', 'a.c.', 'a.g.', 'a.m', 'a.m.', 'a.p', 'ab', 'aba', 'abandon', 'abandoned', 'abandoning', 'abbie', 'abc', 'ability', 'able', 'abm', 'aboard', 'abolish', 'abolished', 'aborted', 'abortion', 'abortion-rights', 'abortions', 'about', 'above', 'abrams', 'abramson', 'abroad', 'abrupt', 'abruptly', 'absence', 'absolutely', 'absorb', 'absorbed', 'absurd', 'abundant', 'abuse', 'abused', 'abuses', 'academic', 'academy', 'acadia', 'accelerate', 'accelerated', 'accelerating', 'acceleration', 'accept', 'acceptable',

In [7]:
from gensim.models import Word2Vec
model1 = Word2Vec(train_data, min_count=1, size=100, window=5)
model1.train(train_data, len(train_data), epochs=100, total_examples=model1.corpus_count)

(66888001, 97292600)

In [8]:
model1.wv.most_similar("stock")

[('shares', 0.6012330055236816),
 ('stocks', 0.56917405128479),
 ('equity', 0.49792689085006714),
 ('share', 0.49605488777160645),
 ('mercantile', 0.4488602876663208),
 ('junk', 0.44790762662887573),
 ('sterling', 0.42159849405288696),
 ('junk-bond', 0.4125289022922516),
 ('options', 0.4084639549255371),
 ('plunge', 0.4042600989341736)]

In [9]:
model1.wv.most_similar(".")

[('f.', 0.49283674359321594),
 ('e.', 0.4552537202835083),
 ('h.', 0.44416117668151855),
 ('w.', 0.4380009174346924),
 ('s.', 0.3974824547767639),
 ('l.', 0.39728131890296936),
 ('p.', 0.3883320987224579),
 ('wash.', 0.384010374546051),
 ('fluor', 0.37740230560302734),
 ('n.j.', 0.3744466304779053)]

In [10]:
model1.wv.most_similar("credit")

[('loans', 0.4509538412094116),
 ('loan', 0.42039382457733154),
 ('loan-loss', 0.41511422395706177),
 ('financial', 0.4025123119354248),
 ('debt', 0.3996398150920868),
 ('allowance', 0.396798700094223),
 ('receivables', 0.39609915018081665),
 ('credit-card', 0.3795093595981598),
 ('home-equity', 0.3659972548484802),
 ('dai-ichi', 0.36121222376823425)]

In [11]:
model1.wv['credit']

array([ 1.8408158 ,  2.1528978 ,  1.577311  , -0.5424135 , -0.77825797,
       -3.487012  , -4.1693244 ,  1.2087706 ,  2.6357746 , -1.2623004 ,
        1.9656764 ,  3.674057  , -0.04240939, -0.88878816,  1.9305915 ,
        0.50211966,  1.0581901 ,  1.0795053 , -0.74899954,  2.7184741 ,
       -2.354455  , -2.7535293 ,  4.412339  ,  1.6338178 , -1.503997  ,
       -1.8006854 ,  0.9367318 ,  5.7195153 , -3.2364767 ,  0.9345901 ,
       -1.8378781 , -0.6263213 ,  2.9540033 ,  2.7205722 ,  1.0509466 ,
        0.5784444 , -4.2308373 ,  3.083613  ,  0.39906096,  0.20975357,
        0.84889466, -0.6421074 ,  0.50029224,  1.4838523 , -3.8019986 ,
       -0.8341925 ,  1.4484494 ,  1.0791607 ,  2.0743005 ,  0.58189577,
        1.509926  ,  0.1290245 , -1.772432  , -0.22600487,  0.52499324,
       -0.19749175, -1.8292658 ,  1.5739354 ,  0.9197604 ,  2.1917448 ,
       -0.4205204 , -0.8827047 , -1.0129049 ,  0.5440537 , -1.0531436 ,
       -1.1322188 ,  3.1973696 , -4.4078317 ,  1.8610548 , -1.50

In [18]:
vocabulary_size = len(vocabulary)
input_size = 100
output_size = 100
hidden_size = 10

In [13]:
input_word = 'money'
embedding_vec = model1.wv[input_word]

In [50]:
# make the word embeddings into a pythorch tensor
weights = torch.FloatTensor(model1.wv.vectors)

# NN MODEL
embedded = nn.Embedding.from_pretrained(weights) #input layer
lstm = nn.LSTM(input_size, hidden_size, num_layers=1) #lstm layer
out = nn.Linear(hidden_size, output_size)
softmax = nn.LogSoftmax(dim=1)

# loss_function = nn.NLLLoss()
# optimizer = optim.SGD( lr=0.1)

In [51]:
input_sentence = train_data[2]
input_idx = [model1.wv.vocab[x].index for x in input_sentence]

In [52]:
# intialise first hidden state
(hidden, cell) = (torch.zeros(1, 1, hidden_size), torch.zeros(1, 1, hidden_size))

for idx in input_idx:
    print("input word:", model1.wv.index2word[idx])
    output = embedded(torch.tensor([idx], dtype=torch.long)).view(1, 1, 100)
    output, (hidden, cell) = lstm(output, (hidden, cell))
    output = softmax(out(output[0]))
    predicted_idx = np.argmax(output.detach().numpy())
    print("predicted_class: {0}".format(model1.wv.index2word[predicted_idx]))

input word: <sos>
predicted_class: <eos>
input word: mr.
predicted_class: million
input word: <unk>
predicted_class: 's
input word: is
predicted_class: its
input word: chairman
predicted_class: inc.
input word: of
predicted_class: after
input word: <unk>
predicted_class: when
input word: n.v.
predicted_class: if
input word: the
predicted_class: corp.
input word: dutch
predicted_class: have
input word: publishing
predicted_class: sales
input word: group
predicted_class: after
input word: <eos>
predicted_class: after


In [142]:
class rnn_language_model(nn.Module):
    def __init__(self, vocabulary_size, input_size, hidden_size, num_layers, weights):
        super(rnn_language_model, self).__init__()
        self.embed = nn.Embedding.from_pretrained(weights)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, vocabulary_size)
        
    def forward(self, x, h):
        x = self.embed(torch.tensor(x, dtype=torch.long)).view(1, len(x), 100)
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

In [172]:
learning_rate = 0.0001
num_layers = 1
batch_size = 100
epochs = 100

model = rnn_language_model(vocabulary_size, input_size, hidden_size, num_layers, weights)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Fixed batch size - this should probably be changed later since the size will vary from sentence to sentence, not sure how...
def get_batch(data, batch_size):
    batches = []
    count = 0
    current_batch = []
    for i in range(len(data)):
        for j in range(len(data[i])):
            if count == batch_size + 1:
                batches.append(current_batch)
                current_batch = []
                count = 0
            else:
                current_batch.append(data[i][j])
                count += 1
    return batches  

batches = get_batch(train_data[1:100], batch_size)
print(len(batches))

21


In [ ]:
# Train
for epoch in range(epochs):
    (hidden, cell) = (torch.zeros(num_layers, batch_size, hidden_size), torch.zeros(num_layers, batch_size, hidden_size))
    for i in range(len(batches)):
        #for j in range(len(train_data[i]) - 1):
        inputs = [model1.wv.vocab[x].index for x in batches[i][:-1]]
        targets = torch.tensor([model1.wv.vocab[x].index for x in batches[i][1:]], dtype=torch.long)

        hidden = hidden.detach()
        cell = cell.detach()
    
        outputs, (hidden, cell) = model(inputs, (hidden, cell))
        loss = criterion(outputs, targets)

        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

    if epoch % 1 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
               .format(epoch+1, epochs, loss.item(), np.exp(loss.item())))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Epoch [1/100], Loss: 9.1851, Perplexity: 9750.43
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Epoch [2/100], Loss: 9.1781, Perplexity: 9682.86
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Epoch [3/100], Loss: 9.1711, Perplexity: 9615.25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
